# SSH Port Forwarding

# code block
(base) mint20@mint20:~$ ssh -L 8069:192.168.25.212:8069 localhost

The authenticity of host 'localhost (127.0.0.1)' can't be established.
ECDSA key fingerprint is SHA256:PhJlLRGB1s9Z7Dz+xPH1knqNuu6aq8sYM1DEnGRQUMM.
Are you sure you want to continue connecting (yes/no/[fingerprint])? yes
Warning: Permanently added 'localhost' (ECDSA) to the list of known hosts.
mint20@localhost's password: 

# code block
(base) mint20@mint20:~$ ssh -L 8069:192.168.25.212:8069 root@192.168.25.212

root@192.168.25.212's password: 
Welcome to Tkl-odoo16-vm-18, TurnKey GNU/Linux 18.0 (Debian 12/Bookworm)

  System information for Wed Nov 13 15:04:46 2024 - -03 (UTC-0300)
  
    System load:  0.30               Memory usage:  18.7%
    Processes:    207                Swap usage:    3.4%
    Usage of /:   13.3% of 29.36GB   IP address for eth0: 192.168.25.212
  
  TKLBAM (Backup and Migration):  NOT INITIALIZED
  
    To initialize TKLBAM, run the "tklbam-init" command to link this
    system to your TurnKey Hub account. For details see the man page or
    go to:
  
        https://www.turnkeylinux.org/tklbam
  

    For Advanced commandline config run:    confconsole

  For more info see: https://www.turnkeylinux.org/docs/confconsole

Linux tkl-odoo16-vm-18 6.1.0-26-amd64 #1 SMP PREEMPT_DYNAMIC Debian 6.1.112-1 (2024-09-30) x86_64
Last login: Wed Nov 13 10:34:58 2024 from 192.168.25.106
root@tkl-odoo16-vm-18 ~#


# Testes Iniciais

## Leitura das Credenciais

In [1]:
import pandas as pd

hosts = pd.read_csv('data/host.csv', delimiter=';', index_col='name')
def creds(index, col_name):
    return hosts.at[index, col_name]

# creds = lambda index, col_name : hosts.at[index, col_name]

# print(hosts)

## Parâmetros globais

In [2]:
import logging

logging.basicConfig(level=logging.DEBUG)
_logger = logging.getLogger(__name__)


In [3]:
# import ssl

# ssl._create_default_https_context = ssl._create_unverified_context
# ctx = ssl.create_default_context()
# ctx.check_hostname = False
# ctx.verify_mode = ssl.CERT_NONE


In [4]:
host_name = 'tkl-odoo16-vm-18'

server = creds(host_name, 'server')
master_pw = creds(host_name, 'master_pw')
dbname = creds(host_name, 'dbname')
user = creds(host_name, 'user')
user_pw = creds(host_name, 'user_pw')
user_apikey = creds(host_name, 'user_apikey')

_logger.info('host_name: "{0}"'.format(host_name))

# print(server_url)


INFO:__main__:host_name: "tkl-odoo16-vm-18"


## Teste de conexão à Base de dados

In [5]:
import erppeek
import xmlrpc.client as xmlrpclib

url = server
sock_common = xmlrpclib.ServerProxy('{}/xmlrpc/2/common'.format(url))
_logger.info('sock_common.version(): "{0}"'.format(sock_common.version()))

client = erppeek.Client(server=server)
_logger.info('Databases found: {0}'.format(client.db.list()))


INFO:__main__:sock_common.version(): "{'server_version': '16.0', 'server_version_info': [16, 0, 0, 'final', 0, ''], 'server_serie': '16.0', 'protocol_version': 1}"
INFO:__main__:Databases found: ['clvhealth_jcafb_16']


## Teste de 'clvsol_odoo_client2.host.host_login'

In [6]:
from clvsol_odoo_client2 import host

host.host_login(server, dbname, user, user_pw)


INFO:clvsol_odoo_client2.host:--> host_login http://192.168.25.212:8069 clvhealth_jcafb_16
INFO:clvsol_odoo_client2.host:--> uid 2
INFO:clvsol_odoo_client2.host:--> sock <ServerProxy for 192.168.25.212:8069/xmlrpc/2/object>
INFO:clvsol_odoo_client2.host:--> [01] Login Ok. Administrator 
INFO:clvsol_odoo_client2.host:--> {'id': 2, 'name': 'Administrator', 'parent_id': False}


(2, <ServerProxy for 192.168.25.212:8069/xmlrpc/2/object>, '[01] Login Ok.')

In [7]:
host.host_login(server + 'x', dbname, user, user_pw)


INFO:clvsol_odoo_client2.host:--> host_login http://192.168.25.212:8069x clvhealth_jcafb_16
INFO:clvsol_odoo_client2.host:--> [11] Server is not responding.  


(False, False, '[11] Server is not responding.')

In [8]:
host.host_login(server, dbname + 'x', user, user_pw)

INFO:clvsol_odoo_client2.host:--> host_login http://192.168.25.212:8069 clvhealth_jcafb_16x
INFO:clvsol_odoo_client2.host:--> except Fault
INFO:clvsol_odoo_client2.host:--> [22] Database does not exist.  


(False, False, '[22] Database does not exist.')

In [9]:
host.host_login(server, dbname, user + 'x', user_pw)


INFO:clvsol_odoo_client2.host:--> host_login http://192.168.25.212:8069 clvhealth_jcafb_16
INFO:clvsol_odoo_client2.host:--> uid False
INFO:clvsol_odoo_client2.host:--> sock <ServerProxy for 192.168.25.212:8069/xmlrpc/2/object>
INFO:clvsol_odoo_client2.host:--> [21] Invalid Login/Pasword.  


(False,
 <ServerProxy for 192.168.25.212:8069/xmlrpc/2/object>,
 '[21] Invalid Login/Pasword.')

In [10]:
host.host_login(server, dbname, user, user_pw + 'x')

INFO:clvsol_odoo_client2.host:--> host_login http://192.168.25.212:8069 clvhealth_jcafb_16
INFO:clvsol_odoo_client2.host:--> uid False
INFO:clvsol_odoo_client2.host:--> sock <ServerProxy for 192.168.25.212:8069/xmlrpc/2/object>
INFO:clvsol_odoo_client2.host:--> [21] Invalid Login/Pasword.  


(False,
 <ServerProxy for 192.168.25.212:8069/xmlrpc/2/object>,
 '[21] Invalid Login/Pasword.')

## Teste de 'CLVhealthJCAFB'

In [11]:
from clvhealth_jcafb import *

clvhealth = CLVhealthJCAFB(server=server,
                           admin_user_pw=user_pw,
                           super_user_pw=master_pw,
                           dbname=dbname
                          )

# _logger.info('clvhealth_jcafb.server: {0}'.format(clvhealth.server))
# _logger.info('clvhealth_jcafb.dbname: {0}'.format(clvhealth.dbname))
# _logger.info('clvhealth_jcafb.db.server: {0}'.format(clvhealth.db.server))
# _logger.info('clvhealth_jcafb.db.dbname: {0}'.format(clvhealth.db.dbname))

clvhealth.install()


INFO:clvsol_odoo_client2.db:--> sock_common.version(): "{'server_version': '16.0', 'server_version_info': [16, 0, 0, 'final', 0, ''], 'server_serie': '16.0', 'protocol_version': 1}"
INFO:clvsol_odoo_client2.db:--> Databases found: "['clvhealth_jcafb_16']"
INFO:clvsol_odoo_client2.db:--> Database "clvhealth_jcafb_16" already exists.
INFO:clvsol_odoo_client2.db:--> Done.


--> create_database()

--> newDB:  False

--> my_company_setup()
Configuring My Company...
Done.

--> Administrator()
Configuring user "Administrator"...
Executing user_groups_setup...
Done (user_groups_setup).
Done.

--> data_administrator_user_setup()
Configuring user "Data Administrator"...
Done.

--> newDB:  False

--> Update Modules List"
